# LGBM - Median Gap

In [1]:
import numpy as np
import pandas as pd
from utils import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import sys
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime

NUMBER_OF_LAGS = 4

sys.path.append("../../main/datasets/")
!ls  ../../main/datasets/

1.0v.zip


<hr>

## Defining metrics

Baseline_score function

In [2]:
def baseline_score(prediction, target, simulatedPrice):
    prediction = prediction.astype(int)

    return np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice)

Evaluation Metric

In [3]:
def feval(prediction, dtrain):
    
    prediction = prediction.astype(int)
    target = dtrain.get_label()

    simulatedPrice = dtrain.get_weight()
    
    return 'feval', np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice), True

Objective Metric

In [4]:
def gradient(predt, dtrain):
    y = dtrain.get_label()
    sp = dtrain.get_weight()
    return -2 * (predt - np.maximum(predt - y, 0) * 1.6) * (1 - (predt > y) * 1.6) * sp

def hessian(predt, dtrain):
    y = dtrain.get_label()
    sp = dtrain.get_weight() 
    return -2 * ((1 - (predt > y) * 1.6) ** 2) * sp

def objective(predt, dtrain):
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess

<hr>

## Building our dataset
This notebook makes this step cleaner than the previous versions. So It'll be tidier and shorter than before!

In [5]:
infos, items, orders = read_data("../../main/datasets/")
print("Sanity checks...", infos.shape, items.shape, orders.shape)

Sanity checks... (10463, 3) (10463, 8) (2181955, 5)


In [6]:
# Changing our time signatures
process_time(orders)

In [7]:
df = dataset_builder(orders, items)

<hr>

## Feature building

In [8]:
# percentage_accum_cat_3 feature...
df = cumulative_sale_by_category(df)

In [9]:
# Encoding our weeks as a series of sines and cosines...
# This function will consider our period as a semester in a year,
# so we can try other types of time encoding later!
df = time_encoder(df, 'group_backwards', 26)

In [10]:
# This cell lags and diffs our feature 'orderSum'
shifting = df.copy()

for i in range(1, NUMBER_OF_LAGS + 1):
    # Carrying the data of weeks t-1
    shifting[f'orderSum_{i}'] = shifting.groupby('itemID')['orderSum'].shift(i)

    
    # Getting the difference of the orders and promotions between weeks t-1 and t-2...
    shifting[f'orderSum_diff_{i}'] = shifting.groupby('itemID')[f'orderSum_{i}'].diff()
    

In [11]:
# This cell creates rolling-window features based on 'orderSum' in our dataset!

item_group = shifting.groupby(["itemID", "group_backwards"]).agg({'orderSum':'sum'})

# We'll .shift(-1) because it sorts our "group_backwards", 
# so doing .shift(1) would cause a HUGE dataleak.
aux_shifting = item_group.groupby('itemID')[['orderSum']].shift(-1)

aux_shifting.sort_values(['itemID', 'group_backwards'], ascending=[True, False], inplace=True)

for i in range(3):
    rolled_window = aux_shifting.groupby(['itemID'], as_index=False)[['orderSum']].rolling(2 ** i).mean()
    rolled_window.rename(columns={'orderSum':f"orderSum_mean_rolled_{i}"}, inplace=True)
    shifting = pd.merge(shifting, rolled_window, left_on=['itemID', 'group_backwards'], right_on=['itemID', 'group_backwards'])

In [12]:
aux = pd.DataFrame()
for i in range(13, 0, -1):
    accum = shifting.loc[df.group_backwards >= i].groupby(['itemID']).agg(orderSum_median=('orderSum','median'))
    accum['group_backwards'] = i
    aux = pd.concat([aux, accum])
aux.reset_index(inplace=True)
aux['orderSum_median'] = aux.groupby('itemID')['orderSum_median'].shift(1)
shifting = pd.merge(shifting, aux, left_on=['itemID', 'group_backwards'], right_on=['itemID', 'group_backwards'])
shifting['median_gap'] = shifting['orderSum_1'] - shifting['orderSum_median']

In [13]:
# LGBM Says on docs that it automatically handles zero values as NaN,
# so we'll keep this standard...
shifting.fillna(0, inplace=True)

<hr>

## Maximum error
The maximum error we could get in this dataset would be just guessing the mean of our sales from weeks 1 to 12, and that's what the cell below is computing.

In [16]:
worst_possible_prediction = shifting.loc[shifting.group_backwards < 13]['orderSum'].mean()
prediction = np.full(shifting.loc[shifting.group_backwards == 13]['orderSum'].shape, worst_possible_prediction) # Array filled with the mean...
target = shifting.loc[shifting.group_backwards == 13]['orderSum']
print("Guessing the mean of 'orderSum' for all items in target", mse(target, prediction) ** 0.5)

Guessing the mean of 'orderSum' for all items in target 90.29706562119341


<hr>

## Dataset Splitting (Train until week 3 / Val. week 2/ Test week 1)
All my experiments will use weeks 13 to 3 as a train set, week 2 as our validation set and week 1 as a test set.

In [17]:
train = shifting.loc[shifting.group_backwards >= 3]
val = shifting.loc[shifting.group_backwards == 2]
test = shifting.loc[shifting.group_backwards == 1]

weights = infos.set_index('itemID')['simulationPrice'].to_dict()

w_train = train['itemID'].map(weights)
w_val = val['itemID'].map(weights)

In [18]:
# I recommend to the other members of the team keeping the
# datatypes of our datasets as Pandas DataFrames instead of Numpy,
# since It will easier to use Boosting Analysis frameworks
y_train = train['orderSum']
y_val = val['orderSum']
X_train = train.drop(columns=["orderSum"])
X_val = val.drop(columns=["orderSum"])

In [44]:
params = {
#           "objective" : "poisson",
          "objective" : "l1",
          "metric" :"rmse",
          "learning_rate" : 0.5,
          'verbosity': 1,
          'max_depth': 6,
          'num_leaves': 128,
          "min_data_in_leaf":2000,
         }

lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train)
lgbvalid = lgb.Dataset(X_val, label = y_val, weight=w_val)

num_round = 1000
model = lgb.train(params,
                  lgbtrain,
                  num_round,
                  valid_sets = [lgbtrain, lgbvalid], 
                  verbose_eval=5,
                  early_stopping_rounds=5,
#                   fobj=objective,
                  feval=feval,
                  
                 )

Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 40.288	training's feval: 619660	valid_1's rmse: 43.608	valid_1's feval: 83767.3
[10]	training's rmse: 40.2785	training's feval: 675428	valid_1's rmse: 43.5953	valid_1's feval: 88861.4
[15]	training's rmse: 40.2766	training's feval: 679409	valid_1's rmse: 43.5926	valid_1's feval: 90508.6
Early stopping, best iteration is:
[13]	training's rmse: 40.2775	training's feval: 682545	valid_1's rmse: 43.5943	valid_1's feval: 91170.9


<hr>

## Dataset Splitting (Train until week 2 and test with week 1)
All my experiments will use weeks 13 to 2 as a train set and week 1 as test

In [36]:
train = shifting.loc[shifting.group_backwards >= 2]
val = shifting.loc[shifting.group_backwards == 1]
test = shifting.loc[shifting.group_backwards == 1]

weights = infos.set_index('itemID')['simulationPrice'].to_dict()

w_train = train['itemID'].map(weights)
w_val = val['itemID'].map(weights)

In [37]:
# I recommend to the other members of the team keeping the
# datatypes of our datasets as Pandas DataFrames instead of Numpy,
# since It will easier to use Boosting Analysis frameworks
y_train = train['orderSum']
y_val = val['orderSum']
X_train = train.drop(columns=["orderSum"])
X_val = val.drop(columns=["orderSum"])

In [43]:
params = {
#           "objective" : "poisson",
          "objective" : "l1",
          "metric" :"rmse",
          "learning_rate" : 0.5,
          'verbosity': 1,
          'max_depth': 6,
          'num_leaves': 128,
          "min_data_in_leaf":2000,
         }

lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train)
lgbvalid = lgb.Dataset(X_val, label = y_val, weight=w_val)

num_round = 1000
model = lgb.train(params,
                  lgbtrain,
                  num_round,
                  valid_sets = [lgbtrain, lgbvalid], 
                  verbose_eval=5,
                  early_stopping_rounds=5,
#                   fobj=objective,
                  feval=feval,
                  
                 )

Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 40.288	training's feval: 619660	valid_1's rmse: 43.608	valid_1's feval: 83767.3
[10]	training's rmse: 40.2785	training's feval: 675428	valid_1's rmse: 43.5953	valid_1's feval: 88861.4
[15]	training's rmse: 40.2766	training's feval: 679409	valid_1's rmse: 43.5926	valid_1's feval: 90508.6
Early stopping, best iteration is:
[13]	training's rmse: 40.2775	training's feval: 682545	valid_1's rmse: 43.5943	valid_1's feval: 91170.9


<hr>

### Utilities

**Predicting at test time**

In [97]:
y_test = test['orderSum']
X_test = test.drop(columns=["orderSum"])
final_predictions = model.predict(X_test)

In [98]:
final_predictions

array([0.27124023, 1.        , 0.5       , ..., 0.        , 0.        ,
       0.        ])

In [99]:
final_predictions[final_predictions < 0] = 0

**Baseline calculation**

In [100]:
baseline_score(final_predictions, y_test.values, infos['simulationPrice'])

90996.96799999998

**Creating our Kaggle CSV**

In [ ]:
final = pd.Series(0, index=np.arange(1, len(items)+1))
final[items.itemID] = final_predictions.astype(int)

final.to_csv("lgbm_kaggle_df.csv", header=["demandPrediction"],
            index_label="itemID", sep="|")

**Saving our model in disk**

In [ ]:
now = datetime.now().strftime("%d-%m-%Y-%Hh%Mm%Ss")
modelName = 'lgbm-' + now
xgb.save_model(modelName)